In [1]:
import pandas as pd

In [2]:
raw = pd.read_excel('PQ-313.xlsx', usecols = 'A:B')
raw.head()

,Data1,Data2
0,Customer,Robert
1,Quantity,5
2,Price,550
3,Discount,20
4,Tax,30


In [3]:
df = raw.copy()
df['G'] = (df['Data1'] == 'Customer').cumsum()

result = []
for i, group in df.groupby('G'):
    group = group.drop(columns = 'G')
    df = group.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df['DiscountNTax'] = df.filter(regex=r'^(Discount\d*|Tax\d*)$').sum(axis=1)
    df['Total Amount'] = df.apply(lambda x: (x['Price'] - x['DiscountNTax']) * x['Quantity'], axis = 1)
    df = df[['Customer', 'Total Amount']]
    result.append(df)

df = pd.concat(result, ignore_index = True)
df.columns.name = None
df.loc['Total'] = df.drop(columns=['Customer']).sum()
df.loc['Total', 'Customer'] = 'Total Sale'
df

,Customer,Total Amount
0,Robert,2500.0
1,Lisa,3550.0
2,Bruce,5560.0
Total,Total Sale,11610.0


In [4]:
df = raw.copy()
df['G'] = (df['Data1'] == 'Customer').cumsum()

result = []
for _, group in df.groupby('G'):
    
    g = (
        group.drop(columns = 'G')
        .T
        .pipe(lambda x: x.set_axis(x.iloc[0], axis = 1))
        .iloc[1:]
    )
    
    g['DiscountNTax'] = (
        g.filter(regex = r'^(Discount\d*|Tax\d*)$')
         .apply(pd.to_numeric, errors = 'coerce')
         .sum(axis = 1)
    )

    g = (
        g.astype({'Quantity': 'int', 'Price': 'float'})
        .assign(**{'Total Amount': lambda x: (x['Price'] - x['DiscountNTax']) * x['Quantity']})
        .loc[:,['Customer', 'Total Amount']]
    )

    result.append(g)
    
df = pd.concat(result, ignore_index = True)
df = (
    pd.concat([df, df.drop(columns = 'Customer').sum().to_frame().T], ignore_index = True)
      .assign(Customer = lambda x: x['Customer'].fillna('Total Sale'))
      .rename_axis(None, axis = 1)
      .reset_index(drop = True)
)
df

,Customer,Total Amount
0,Robert,2500.0
1,Lisa,3550.0
2,Bruce,5560.0
3,Total Sale,11610.0
